In [2]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

from crewai import Agent, Task, Crew

In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['SERPER_API_KEY'] = os.getenv("SERPER_API_KEY")

from notion import pull_from_notion, set_up_graph

data = pull_from_notion()
db = set_up_graph(data)

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


Connected to Notion: Recipe
Connected to Notion: Ingredient


In [3]:
# db = connect_your_graph_db()

# from notion import pull_from_notion, set_up_graph

# data = pull_from_notion()
# conn = set_up_graph(data)

# from agent import SimilarRecipesTool

# sim_tool = SimilarRecipesTool(conn=conn)

from crewai_tools import tool
from graph import get_similar_recipes

@tool("sim_tool")
def sim_tool(db=db, names):
    """
    Given one or more recipe names, return a list of the most
    ingredient-similar recipes for each input.

    db: database connector object (defaults to db)
    names: List[str] containing names of recipes

    """
    return get_similar_recipes(db, names)

recommender = Agent(
    role       = "Recipe Recommender",
    goal       = "Create a weekly dinner menu using similar recipes so that the grocery bill and food waste is minimized, but there is diversity in the meals",
    backstory  = "You know everything about flavour pairings.",
    tools      = [sim_tool]
)

task = Task(
    description = "User likes Cherry Tomato Pasta and Steak + Baked Potato. Suggest 3 "
                  "similar recipes they should try this week.",
    expected_output = "A friendly bulleted list with reasons.",
    agent=recommender
)

crew = Crew(agents=[recommender], tasks=[task])
crew.kickoff()


2025-06-22 13:29:16,943 - 8219691072 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 

I encountered an error while trying to use the tool. This was the error: sim_tool() got an unexpected keyword argument 'recipes'.
 Tool sim_tool accepts these inputs: sim_tool() -      Given one or more recipe names, return a list of the most     ingredient-similar recipes for each input.     

 

I encountered an error while trying to use the tool. This was the error: sim_tool() got an unexpected keyword argument 'recipes'.
 Tool sim_tool accepts these inputs: sim_tool() -      Given one or more recipe names, return a list of the most     ingredient-similar recipes for each input.     

 

I encountered an error while trying to use the tool. This was the error: sim_tool() got an unexpected keyword argument 'recipe_names'.
 Tool sim_tool accepts these inputs: sim_tool() -      Given one or more recipe names, return a list of the most     ingredient-similar recipes for each input.     

 

I encountered an error while trying to use the tool. This was the error: sim_tool() got an unex

KeyboardInterrupt: 

In [9]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
# read_resume = FileReadTool(file_path='./fake_resume.md')
# semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

In [29]:
# Agent 1: Personal Chef
personal_chef = Agent(
    role="Personal Chef",
    goal="Make sure to plan a week of dinners focusing on whole foods and protein rich ingredients ",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        """
        As a personal chef, you are an expert in crafting delicous healthy dinner menus.
        Your skills help you build a cohesive weekly dinner plan for a young active couple. 
        Each meal should contain a protein, carb and veggies, be satiating and comprised of primarily whole ingredients.
        """
    )
)

In [30]:
# Agent 2: Meal Optimizer
optimizer = Agent(
    role="Menu Optimizer",
    goal="""
        Refine the meunu to repeatedly use similar ingredients
        in new and creative ways through the week keep the clients enjoying new meals 
        each night without overspending at the grocery store.
        """,
    verbose=True,
    backstory=(
        """
        Your understanding of cooking  techniques and recipes helps you to determine appropriate replacements for ingredients
        that still maintain the overall composition of the menu. Your judgement in determining when to swap and ingredient for another 
        helps you to maintain to quality of the menu and balance the cost.
        """
    )
)

In [32]:
# Agent 3: list formater
formater = Agent(
    role="Professional Grocery Shopper",
    goal="""
        Create a grocery list that combines that menu ingredients for the week into one easily consumable list to take shopping
        """,
    verbose=True,
    backstory=(
        """
        Your understanding of popular grocery store layouts and operations makes you the best at crafting an ordered checklist to take 
        to the grocery store to have the most efficient experience shopping.
        """
    )
)

In [22]:
# Task for Personal Chef Agent
menu_task = Task(
    description=(
        """
        Create a menu of 7 meals that can be eaten for dinner
        """
    ),
    expected_output=(
        """
        JSON object containing the recipe title, the ingredients and the respective quantities per ingredient
        """
    ),
    agent=personal_chef,
    async_execution=True
)

In [23]:
# Task for Optimizer Agent
optimize_menu = Task(
    description=(
        """
        Review the provided menu for the week. Determine which ingredients or recipes should be replaced to bring down the total cost of the grocery run. 
        """
    ),
    expected_output=(
        """
        JSON containing the recipe title, the ingredients and the respective quantities per ingredient
        """
    ),
    allow_delegation=True,
    agent=optimizer,
    async_execution=True
)

In [33]:
# Task for Optimizer Agent
grocery_list = Task(
    description=(
        """
        Extract the ingredients from the provided JSON to create a grocery list, 
        sum the total required ingredients so that each ingredient has a single quantity 
        """
    ),
    expected_output=(
        """
        Checklist of ingredients and the quantities to purchase at the store
        """
    ),
    agent=formater,
    async_execution=True
)

In [34]:
meal_crew = Crew(
    agents=[personal_chef, optimizer, formater],
    tasks=[menu_task, optimize_menu, grocery_list],
    verbose=True
)

2025-01-20 12:46:57,291 - 8265531968 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [35]:
### this execution will take a few minutes to run
result = meal_crew.kickoff()

print('####################')
print(result)

 [DEBUG]: == Working Agent: Personal Chef
 [INFO]: == Starting Task: 
        Create a menu of 7 dinner meals
        
 [DEBUG]: == [Personal Chef] Task output: 


 [DEBUG]: == Working Agent: Menu Optimizer
 [INFO]: == Starting Task: 
        Review the provided menu for the week. Determine which ingredients should be replaced to bring down the total cost of the grocery run. 
        
 [DEBUG]: == [Menu Optimizer] Task output: 


 [DEBUG]: == Working Agent: Professional Grocery Shopper
 [INFO]: == Starting Task: 
        Extract the ingredients from the provided JSON to create a grocery list, 
        sum the total required ingredients so that each ingredient has a single quantity 
        
 [DEBUG]: == [Professional Grocery Shopper] Task output: 


####################





> Entering new CrewAgentExecutor chain...




> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
I need to extract the ingredients from the provided JSON in order to create a grocery list with the total required quantities.

Action: Delegate work to co-worker
Action Input: {"coworker": "Menu Optimizer", "task": "Extract ingredients from JSON", "context": "Here is the JSON file with the menu ingredients"}

> Entering new CrewAgentExecutor chain...
I need to carefully review the menu for the week and identify ingredients that can be replaced without compromising the overall composition of the dishes.

Action: Delegate work to co-worker
Action Input: {"coworker": "Menu Optimizer", "task": "Review the menu for the week and identify ingredients that can be replaced to bring down the total cost of the grocery run.", "context": "Provided menu for the week"} 


Error executing tool. Co-worker mentioned not found, it must to be one of the following options:
- personal chef


I need to gather recipes for

In [27]:
print(result)